**Assignment 1**

Use the following embedding techniques and make an observation on the results for the query in the notebook



1.   BAAI/bge-small-en-v1.5
2.   all-MiniLM-L6-v2
3. sentence-transformers/all-MiniLM-L12-v2

with document split sizes 300, 500 and 1000. Form a tabular column and check which gives the best answer





you can also try other embedding techniques in the mteb leaderboard

https://huggingface.co/spaces/mteb/leaderboard

**Caution:** Free version of Colab has got memory restrictions and may crash for bigger models




# Steps to build a RAG from scratch
### Step-1: Import ``` ChatTogether ``` from ``` langchain_together ```

In [1]:
! pip install langchain-together

In [2]:
from langchain_together import ChatTogether

In [3]:
api_key="f3d79b54977743aa7920cb3434b7729d2cc89b3ee499f1e3327b246234869d4d"

### Step-3: Define your model

In [4]:
!pip install langchain_community

In [5]:
!pip install pypdf

### Step-6: Load the pdf for context

In [6]:
! pip install docarray

In [7]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")

<ipython-input-7-e4a653ae12bb>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

In [9]:
from langchain_community.vectorstores import DocArrayInMemorySearch # Add this line
from langchain_community.document_loaders import PyPDFLoader # Import PyPDFLoader

from langchain_community.vectorstores import DocArrayInMemorySearch # Add this line

loader=PyPDFLoader("VITEEE-2024-information-brochure.pdf")
pages=loader.load_and_split()



In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", ";", ",", " ", ""],
)
page_split = text_splitter.split_documents(pages)

In [11]:
vectorstore = DocArrayInMemorySearch.from_documents(
    page_split,
    embedding=embedding_function
)

retriever=vectorstore.as_retriever()


/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [12]:
from langchain.prompts import PromptTemplate

template="""
    Answer the question based on the context below. If you don't know the answer, just say so.
    Context: {context}
    Question: {question}
"""
prompt=PromptTemplate.from_template(template)

In [13]:
MODEL = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

model = ChatTogether(
    together_api_key=api_key,
    model=MODEL,
)



### Step-10: Defining the chain for the RAG

In [14]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [15]:
from operator import itemgetter
chain = (
    {
        "context":itemgetter("question") | retriever,
        "question":itemgetter("question")
    }
    | prompt
    | model
    | parser
)

In [16]:
print(chain.invoke({"question":"What is the age limit for the applicants?"}))

The provided context does not explicitly mention the age limit for the applicants. It only mentions that the date of birth as recorded in the High School / SSC / X Certificate will be considered authentic and applicants should produce this certificate in original as proof of their age at the time of counselling/admission.
